In [1]:
import datetime, time
import simpy

import shapely.geometry


import pandas as pd
import openclsim.core as core
import openclsim.model as model
import openclsim.plot as plot
import openclsim.plugins as plugins

# setup environment
simulation_start = 0
my_env = simpy.Environment(initial_time=simulation_start)
registry = {}
keep_resources = {}

In [2]:
# The generic site class
Site = type(
    "Site",
    (
        core.Identifiable,  # Give it a name
        core.Log,  # Allow logging of all discrete events
        core.Locatable,  # Add coordinates to extract distance information and visualize
        core.HasContainer,  # Add information on the material available at the site
        core.HasResource,
    ),  # Add information on serving equipment
    {},
)  # The dictionary is empty because the site type is generic

# Information on the extraction site - the "from site" - the "win locatie"
location_from_site = shapely.geometry.Point(4.18055556, 52.18664444)  # lon, lat

data_from_site = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Winlocatie",  # The name of the site
    "ID": "6dbbbdf4-4589-11e9-a501-b469212bff5d",  # For logging purposes
    "geometry": location_from_site,  # The coordinates of the project site
    "capacity": 12,  # The capacity of the site
    "level": 12,
}  # The actual volume of the site

# Information on the dumping site - the "to site" - the "dump locatie"
location_to_site = shapely.geometry.Point(4.25222222, 52.11428333)  # lon, lat

data_to_site = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Dumplocatie",  # The name of the site
    "ID": "6dbbbdf5-4589-11e9-82b2-b469212bff5c",  # For logging purposes
    "geometry": location_to_site,  # The coordinates of the project site
    "capacity": 12,  # The capacity of the site
    "level": 0,
}  # The actual volume of the site (empty of course)

# The two objects used for the simulation
from_site = Site(**data_from_site)
to_site = Site(**data_to_site)

In [3]:
# The generic class for an object that can move and transport (a TSHD for example)
TransportProcessingResource = type(
    "TransportProcessingResource",
    (
        core.Identifiable,  # Give it a name
        core.Log,  # Allow logging of all discrete events
        core.ContainerDependentMovable,  # A moving container, so capacity and location
        core.Processor,  # Allow for loading and unloading
        core.HasResource,  # Add information on serving equipment
        core.LoadingFunction,  # Add a loading function
        core.UnloadingFunction,  # Add an unloading function
    ),
    {},
)

# For more realistic simulation you might want to have speed dependent on the volume carried by the vessel
def compute_v_provider(v_empty, v_full):
    return lambda x: 10

# TSHD variables
data_hopper = {
    "env": my_env,  # The simpy environment
    "name": "Hopper 01",  # Name
    "ID": "6dbbbdf6-4589-11e9-95a2-b469212bff5b",  # For logging purposes
    "geometry": location_from_site,  # It starts at the "from site"
    "loading_rate": 1,  # Loading rate
    "unloading_rate": 1,  # Unloading rate
    "capacity": 4,  # Capacity of the hopper - "Beunvolume"
    "compute_v": compute_v_provider(5, 4.5),  # Variable speed
}

hopper = TransportProcessingResource(**data_hopper)

## Definition of the activities

In [4]:
DelaySequenceActivity =  type(
    "TestShiftActivity",
    (
        plugins.HasDelayPlugin,
        model.SequentialActivity,  # the order is critical!
    ),
    {},
)

DelayWhileActivity =  type(
    "TestShiftActivity",
    (
        plugins.HasDelayPlugin,
        model.WhileActivity,  # the order is critical!
    ),
    {},
)

DelayMoveActivity =  type(
    "TestMoveActivity",
    (
        plugins.HasDelayPlugin,
        model.MoveActivity,  # the order is critical!
    ),
    {},
)


DelayShiftActivity =  type(
    "TestShiftActivity",
    (
        plugins.HasDelayPlugin,
        model.ShiftAmountActivity,  # the order is critical!
    ),
    {},
)

DelayBasicActivity =  type(
    "TestShiftActivity",
    (
        plugins.HasDelayPlugin,
        model.BasicActivity,  # the order is critical!
    ),
    {},
)

In [5]:
single_run = []

move_activity_to_harbor_data = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "sailing empty",  # We are moving soil
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5d",  # For logging purposes
    "registry": registry,
    "mover": hopper,
    "destination": from_site,
    "postpone_start": True,
    "delay_percentage":10,
}
single_run.append(DelayMoveActivity(**move_activity_to_harbor_data))

shift_amount_activity_loading_data = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Transfer MP",  # We are moving soil
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff52",  # For logging purposes
    "registry": registry,
    "processor": hopper,
    "origin": from_site,
    "destination": hopper,
    "amount": 4,
    "duration": 10,
    "postpone_start": True,
    "delay_percentage":10,
}
single_run.append(DelayShiftActivity(**shift_amount_activity_loading_data))

move_activity_to_site_data = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "sailing filler",  # We are moving soil
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5b",  # For logging purposes
    "registry": registry,
    "mover": hopper,
    "destination": to_site,
    "postpone_start": True,
    "delay_percentage":10,
}
single_run.append(DelayMoveActivity(**move_activity_to_site_data))

shift_amount_activity_unloading_data = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "Transfer TP",  # We are moving soil
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff54",  # For logging purposes
    "registry": registry,
    "processor": hopper,
    "origin": hopper,
    "destination": to_site,
    "amount": 4,
    "duration": 10,
    "postpone_start": True,
    "delay_percentage":10,
}
single_run.append(DelayShiftActivity(**shift_amount_activity_unloading_data))

basic_activity_data = {
    "env": my_env,
    "name": "Basic activity",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5h",  # For logging purposes
    "registry": registry,
    "duration": 0,
     "additional_logs": [hopper],
    "postpone_start": True,
    "delay_percentage":10,
}
single_run.append(DelayBasicActivity(**basic_activity_data))


sequential_activity_data = {
    "env": my_env,
    "name": "Single run process",
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff60",  # For logging purposes
    "registry": registry,
    "sub_processes": single_run,
    "postpone_start": True,
    "delay_percentage":10,
}
activity = DelaySequenceActivity(**sequential_activity_data)

expr = [{"type":"container", "concept": to_site, "state":"full"}]
while_data = {
    "env": my_env,  # The simpy environment defined in the first cel
    "name": "while",  # We are moving soil
    "ID": "6dbbbdf7-4589-11e9-bf3b-b469212bff5g",  # For logging purposes
    "registry": registry,
    "sub_processes": [activity],
    "condition_event": expr,
    "postpone_start": False,
    "delay_percentage":10,
}
while_activity = DelayWhileActivity(**while_data)


In [6]:
my_env.run()

In [7]:
pd.concat([plot.get_log_dataframe(act, [while_activity, activity, *single_run]) for act in single_run]).sort_values(by='Timestamp')

,Activity,Timestamp,ActivityState,type,ref
0,sailing empty,1970-01-01 00:00:00.000000,START,NaN,NaN
1,sailing empty,1970-01-01 00:00:00.000000,STOP,NaN,NaN
2,sailing empty,1970-01-01 00:00:00.000000,WAIT_START,plugin,delay
3,sailing empty,1970-01-01 00:00:00.000000,WAIT_STOP,plugin,delay
0,Transfer MP,1970-01-01 00:00:00.000000,START,NaN,NaN
1,Transfer MP,1970-01-01 00:00:10.000000,STOP,NaN,NaN
2,Transfer MP,1970-01-01 00:00:10.000000,WAIT_START,plugin,delay
3,Transfer MP,1970-01-01 00:00:11.000000,WAIT_STOP,plugin,delay
0,sailing filler,1970-01-01 00:00:11.000000,START,NaN,NaN
2,sailing filler,1970-01-01 00:15:53.824591,WAIT_START,plugin,delay


In [8]:
objects = [hopper, from_site, to_site, while_activity, activity]
objects.extend(single_run)
plot.vessel_planning(objects)